# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

from pprint import pprint

# Configure gmaps
gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_df=pd.read_csv('../WeatherPy/weather_data.csv')

weather_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,chuy,-33.6971,-53.4616,77.14,68.0,2.0,7.61,UY,1.611955e+09
1,rikitea,-23.1203,-134.9692,79.88,67.0,97.0,6.49,PF,1.611955e+09
2,yellowknife,62.4560,-114.3525,-14.01,76.0,1.0,9.22,CA,1.611955e+09
3,khuzdar,27.7384,66.6434,47.61,36.0,4.0,2.48,PK,1.611955e+09
4,upernavik,72.7868,-56.1549,-7.31,82.0,86.0,11.95,GL,1.611955e+09
...,...,...,...,...,...,...,...,...,...
602,sorland,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
603,saldanha,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
604,helong,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
605,wladyslawowo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
clean_weather_df=weather_df.dropna(subset=['Lat'])

clean_weather_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,chuy,-33.6971,-53.4616,77.14,68.0,2.0,7.61,UY,1.611955e+09
1,rikitea,-23.1203,-134.9692,79.88,67.0,97.0,6.49,PF,1.611955e+09
2,yellowknife,62.4560,-114.3525,-14.01,76.0,1.0,9.22,CA,1.611955e+09
3,khuzdar,27.7384,66.6434,47.61,36.0,4.0,2.48,PK,1.611955e+09
4,upernavik,72.7868,-56.1549,-7.31,82.0,86.0,11.95,GL,1.611955e+09
...,...,...,...,...,...,...,...,...,...
555,shablykino,67.6670,12.6934,19.40,79.0,40.0,3.44,NO,1.611956e+09
556,husavik,-33.0117,17.9442,65.03,89.0,0.0,12.68,ZA,1.611956e+09
557,hailar,42.5397,128.9972,5.86,90.0,100.0,3.91,CN,1.611956e+09
558,yanchukan,54.7909,18.4009,32.00,93.0,100.0,1.14,PL,1.611956e+09


In [4]:
# Store latitude and longitude in locations
city = clean_weather_df[["Lat", "Lng"]].astype(float)

# Store Humidity and ensure in float format
humidity = clean_weather_df["Humidity"].astype(float)


In [5]:
# Plot Heatmap
fig = gmaps.figure(map_type="HYBRID")

# Create heat layer
heat_layer = gmaps.heatmap_layer(city, weights=humidity, 
                                 dissipating=False,max_intensity=100,
                                 point_radius=1)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
#Create a new dataframe with the ideal weather conditions:
# Humidity less than or equal to 50%
# Max Temperature between 65F and 75F
# Cloudiness <10%

ideal_weather_df=clean_weather_df[(clean_weather_df['Humidity']<=50)&(clean_weather_df['Max Temp']>=65)&(clean_weather_df['Max Temp']<=75)&(clean_weather_df['Cloudiness']<=10)]
ideal_weather_df.describe()

,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Date
count,14.000000,14.000000,14.000000,14.000000,14.000000,14.000000,1.400000e+01
mean,11.080121,6.897914,68.857857,28.785714,1.500000,12.755714,1.611955e+09
std,17.110216,42.225923,3.324578,9.324728,3.057148,5.761805,9.972919e+01
min,-26.648100,-102.077900,65.210000,18.000000,0.000000,4.160000,1.611955e+09
25%,10.090750,9.681025,66.200000,22.250000,0.000000,9.392500,1.611956e+09
50%,13.187150,14.864050,67.740000,25.500000,0.000000,12.035000,1.611956e+09
75%,19.522050,31.169525,72.355000,35.500000,0.750000,15.967500,1.611956e+09
max,32.467400,46.721900,74.280000,46.000000,10.000000,25.480000,1.611956e+09


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
hotel_df=ideal_weather_df.copy()
hotel_df['Hotel Name']=""

In [8]:
# geocoordinates


target_radius = 5000
target_type = "lodging"

# set up a parameters dictionary

query_params = {  
    "radius": target_radius,
    "types": target_type,
    "key": g_key
}

# base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"


for index, row in hotel_df.iterrows():
    lat=row['Lat']
    lng=row['Lng']  
    query_params['location'] = f'{lat},{lng}'
    
    city_name = row['City']
    # assemble url and make API request
    response = requests.get(base_url, params=query_params).json()
  
    # extract results
    results = response['results']
    
    try:
   
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        
    except (KeyError, IndexError):
        pass


In [9]:
hotel_df=hotel_df.loc[hotel_df['Hotel Name']!=""]
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
114,belushya guba,31.9974,-102.0779,66.99,30.0,1.0,11.50,US,1.611955e+09,DoubleTree by Hilton Hotel Midland Plaza
154,shelburne,24.6877,46.7219,66.20,37.0,6.0,7.56,SA,1.611955e+09,InterContinental Riyadh
220,navrongo,17.4924,44.1277,66.20,20.0,0.0,4.16,SA,1.611956e+09,محمد هادي ال فطيح
271,brae,3.5167,39.0584,74.28,31.0,0.0,16.87,KE,1.611955e+09,La Tahzan Guest House
309,padang,-26.6481,15.1594,72.07,39.0,0.0,25.48,NaN,1.611956e+09,LÜDERITZ NEST HOTEL
338,kabanjahe,9.9167,8.9000,65.52,26.0,0.0,8.81,NG,1.611956e+09,Silk Suites
380,buala,10.6129,12.1946,68.49,23.0,0.0,11.88,NG,1.611956e+09,Emir's Palace
388,taltal,-22.0920,-70.1979,72.45,46.0,4.0,4.70,CL,1.611956e+09,Hotel Galvarino
393,rundu,32.4674,14.5687,65.21,44.0,10.0,13.49,LY,1.611956e+09,القسم الداخلي للجامعة الأسمرية
553,liku,13.1629,32.6635,73.27,18.0,0.0,16.40,SD,1.611956e+09,فندق الكوباري - Kobari Hotel


In [10]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [11]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations,
     info_box_content=hotel_info)


# Display figure
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))